In [11]:
%load_ext autoreload
%autoreload 2

from tqdm.notebook import tqdm
import os

from py.Dataset import Dataset
from py.DatasetStatistics import DatasetStatistics
from py.Session import Session
from py.FileUtils import list_folders, list_jpegs_recursive

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [12]:
DIR = '/home/AMMOD_data/camera_traps/BayerWald/Vielkadaver-Projekt/'

ds = Dataset(DIR)

Found 32 sessions


In [13]:
print(ds.get_sessions())

['Beaver_01', 'Marten_01', 'Raccoon_01', 'Reddeer_01', 'Roedeer_01', 'Wildboar_01', 'Badger_02', 'Beaver_02', 'Fox_02', 'Marten_02', 'Raccoon_02', 'Rat_02', 'Reddeer_02', 'Roedeer_02', 'Badger_03', 'Fox_03', 'Raccoon_03', 'Reddeer_03', 'Wildboar_03', 'Badger_04', 'Rat_04', 'Reddeer_04', 'Wildboar_04', 'Badger_05', 'Beaver_05', 'Ermine_05', 'Fox_05', 'Marten_05', 'Raccoon_05', 'Reddeer_05', 'Roedeer_05', 'Wildboar_05']


In [14]:
session = ds.create_session("marten_01")

Session 'Marten_01' at folder: /home/AMMOD_data/camera_traps/BayerWald/Vielkadaver-Projekt/VIELAAS_Spring_Session01-VIELAAS_Marten_01
Session not scanned. Run session.scan() to create scan files
